In [2]:
# Basic Import
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Modelling Imports
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge , Lasso
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

### Load Datasets 

In [3]:
df = pd.read_csv("data/raw.csv")
# showing top 5 records
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [4]:
# Drop the target column 'math_score' 
# Assign all datasets to variable x except the target column
X = df.drop(columns=["math_score"], axis=1)

X

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,74
1,female,group C,some college,standard,completed,90,88
2,female,group B,master's degree,standard,none,95,93
3,male,group A,associate's degree,free/reduced,none,57,44
4,male,group C,some college,standard,none,78,75
...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,99,95
996,male,group C,high school,free/reduced,none,55,55
997,female,group C,high school,free/reduced,completed,71,65
998,female,group D,some college,standard,completed,78,77


In [5]:
print("Categories in 'gender' variable :  ", end=" ")
print(df["gender"].unique())

print("Categories in 'race_ethnicity' variable :  ", end=" ")
print(df["race_ethnicity"].unique())

print("Categories in 'parental_level_of_education' variable :  ", end=" ")
print(df["parental_level_of_education"].unique())

print("Categories in 'lunch' variable :  ", end=" ")
print(df["lunch"].unique())

print("Categories in 'test_preparation_course' variable :  ", end=" ")
print(df["test_preparation_course"].unique())

Categories in 'gender' variable :   ['female' 'male']
Categories in 'race_ethnicity' variable :   ['group B' 'group C' 'group A' 'group D' 'group E']
Categories in 'parental_level_of_education' variable :   ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories in 'lunch' variable :   ['standard' 'free/reduced']
Categories in 'test_preparation_course' variable :   ['none' 'completed']


In [6]:
# Seperate Output Feature 
y = df["math_score"]
y

0      72
1      69
2      90
3      47
4      76
       ..
995    88
996    62
997    59
998    68
999    77
Name: math_score, Length: 1000, dtype: int64

In [7]:
# divide categorical and numerical features

num_feature = X.select_dtypes(exclude="object").columns
cat_feature = X.select_dtypes(include="object").columns

cat_feature

Index(['gender', 'race_ethnicity', 'parental_level_of_education', 'lunch',
       'test_preparation_course'],
      dtype='object')

In [8]:
# Converting Categorical feature to Numerocal Featiure and Scalling Numerical Feature (So we didn't get any outlier values)

from sklearn.preprocessing import OneHotEncoder, StandardScaler
# Here OnehotEncoder is used to convert categorical feature to numerical feature
# StandardScaler is used to scale numerical feature

# if we want to use both OneHotEncoder and StandardScaler then we have to use ColumnTransformer
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
categorical_transformer = OneHotEncoder()


# We are using ColumnTransformer to apply different transformations to different columns
preprocessor = ColumnTransformer(
   [
       ("OneHotEncoder", categorical_transformer, cat_feature),
       ("StandardScaler", numeric_transformer, num_feature)
   ]
)

In [9]:
X = preprocessor.fit_transform(X)

### Here basically convert numerical to standard scalar and categorical to numerical 

In [10]:
X

array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]], shape=(1000, 19))

In [11]:
X.shape

(1000, 19)

In [12]:
# Seperate Datasets into Train and Test
from sklearn.model_selection import train_test_split
# Seperate Datasets into Train and Test with test size of 20% and random state of 42
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((800, 19), (200, 19), (800,), (200,))

## Create an Evalute Function to give all metrics after model training

In [13]:
def evaluate_model(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mse)
    r2 = r2_score(true, predicted)  # ← renamed
    return mae, mse, rmse, r2

In [14]:
models = {
    "Linear Regression": LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "K-Neighbors Regressor": KNeighborsRegressor(),
    "Decision Tree":DecisionTreeRegressor(),
    "Random Forest Regressor": RandomForestRegressor(),
    "XGBRegressor": XGBRegressor(),
    "CatBoosting Regressor": CatBoostRegressor(verbose=False),
    "AdaBoost Regressor": AdaBoostRegressor()
}

model_list = []
r2_list = []

In [15]:
for i in range(len(list(models))):
    model = list(models.values())[i]
    model.fit(X_train , y_train)  # Train Models 

    # Make Predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    # Evaluate Train and Test Datasets
    model_train_mae, model_train_mse, model_train_rmse, model_train_r2 = evaluate_model(
    y_train, y_train_pred
)

    model_test_mae, model_test_mse, model_test_rmse, model_test_r2 = evaluate_model(
    y_test, y_test_pred
)
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model Performance for Training Sets ')
    print("- Root Mean Squared Error : {:.4f}".format(model_train_rmse))
    print("- Mean Absolute Error : {:.4f}".format(model_train_mae))
    print("- R2 Score : {:.4f}".format(model_train_r2))

    print("------------------------------------")

    print('Model Performance for Test Sets ')
    print("- Root Mean Squared Error : {:.4f}".format(model_test_rmse))
    print("- Mean Absolute Error : {:.4f}".format(model_test_mae))
    print("- R2 Score : {:.4f}".format(model_test_r2))

    r2_list.append(model_test_r2)

    print("="*100)
    print("\n")

Linear Regression
Model Performance for Training Sets 
- Root Mean Squared Error : 5.3231
- Mean Absolute Error : 4.2667
- R2 Score : 0.8743
------------------------------------
Model Performance for Test Sets 
- Root Mean Squared Error : 5.3940
- Mean Absolute Error : 4.2148
- R2 Score : 0.8804


Lasso
Model Performance for Training Sets 
- Root Mean Squared Error : 6.5938
- Mean Absolute Error : 5.2063
- R2 Score : 0.8071
------------------------------------
Model Performance for Test Sets 
- Root Mean Squared Error : 6.5197
- Mean Absolute Error : 5.1579
- R2 Score : 0.8253


Ridge
Model Performance for Training Sets 
- Root Mean Squared Error : 5.3233
- Mean Absolute Error : 4.2650
- R2 Score : 0.8743
------------------------------------
Model Performance for Test Sets 
- Root Mean Squared Error : 5.3904
- Mean Absolute Error : 4.2111
- R2 Score : 0.8806


K-Neighbors Regressor
Model Performance for Training Sets 
- Root Mean Squared Error : 5.7172
- Mean Absolute Error : 4.5270
- 

In [17]:
pd.DataFrame(
    list(zip(model_list, r2_list)), columns=["Model Name", "R2_Score"]
).sort_values(by=["R2_Score"], ascending=False)

,Model Name,R2_Score
2,Ridge,0.880593
0,Linear Regression,0.880433
5,Random Forest Regressor,0.854232
8,AdaBoost Regressor,0.853978
7,CatBoosting Regressor,0.851632
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,K-Neighbors Regressor,0.783681
4,Decision Tree,0.746074
